<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>

# Climate Change Satellite Image Classification Competition Model Submission Guide - keras

---
**About the Original Data:**<br>
*Data and Description accessed from [Tensorflow](https://www.tensorflow.org/datasets/catalog/bigearthnet)* <br>
The BigEarthNet is a new large-scale Sentinel-2 benchmark archive, consisting of 590,326 Sentinel-2 image patches. The image patch size on the ground is 1.2 x 1.2 km with variable image size depending on the channel resolution. This is a multi-label dataset with 43 imbalanced labels, which has been simplified to single labels with 3 categories for the purposes of this competition.

To construct the BigEarthNet, 125 Sentinel-2 tiles acquired between June 2017 and May 2018 over the 10 countries (Austria, Belgium, Finland, Ireland, Kosovo, Lithuania, Luxembourg, Portugal, Serbia, Switzerland) of Europe were initially selected. All the tiles were atmospherically corrected by the Sentinel-2 Level 2A product generation and formatting tool (sen2cor). Then, they were divided into 590,326 non-overlapping image patches. Each image patch was annotated by the multiple land-cover classes (i.e., multi-labels) that were provided from the CORINE Land Cover database of the year 2018 (CLC 2018).

Bands and pixel resolution in meters:

    B01: Coastal aerosol; 60m
    B02: Blue; 10m
    B03: Green; 10m
    B04: Red; 10m
    B05: Vegetation red edge; 20m
    B06: Vegetation red edge; 20m
    B07: Vegetation red edge; 20m
    B08: NIR; 10m
    B09: Water vapor; 60m
    B11: SWIR; 20m
    B12: SWIR; 20m
    B8A: Narrow NIR; 20m

License: Community Data License Agreement - Permissive, Version 1.0."

**Competition Data Specifics:**<br>
For the purpose of this competition, the original BigEarthNet dataset has been simplified to 20,000 images (15,000 training images and 5,000 test images) with 3 categories: "forest", "nonforest", and "snow_shadow_cloud", which contains images of snow and clouds. <br>
Each "image" is a folder with 12 satellite image layers, each of which pics up on different features. The example preprocessor uses just three layers: B02, B03, and B04, which contain the standard RGB layers used in ML models. However, you are free to use any combination of the satellite image layers. 

**Data Source:**<br>
Sumbul, G, Charfuelan, M, Demir, B and Markl, V. (2019). BigEarthNet: A Large-Scale Benchmark Archive For Remote Sensing Image Understanding. *Computing Research Repository (CoRR), abs/1902.06148.* https://www.tensorflow.org/datasets/catalog/bigearthnet




# Overview
---

Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data / Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Load Data

In [1]:
!python -V

Python 3.8.0


In [2]:
#install aimodelshare library
#! pip install aimodelshare-nightly

In [3]:
# Get competition data - May take a couple minutes due to size of data set
# from aimodelshare import download_data
# download_data('public.ecr.aws/y2e2a1d6/climate_competition_data-repository:latest') 

In [4]:
# # Unzip Data - May take a couple minutes due to size of data set
# import zipfile
# with zipfile.ZipFile('climate_competition_data/climate_competition_data.zip', 'r') as zip_ref:
#     zip_ref.extractall('competition_data')

##2.   Preprocess data / Write and Save Preprocessor function


In [5]:
# Set up for data preprocessing
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [6]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(imageband_directory):
    """
    This function preprocesses reads in images, resizes them to a fixed shape and
    min/max transforms them before converting feature values to float32 numeric values
    required by onnx files.

    params:
        imageband_directory
            path to folder with 13 satellite image bands

    returns:
        X
            numpy array of preprocessed image data

    """

    import PIL
    import os
    import numpy as np
    import tensorflow_datasets as tfds

    def _load_tif(data):
        """Loads TIF file and returns as float32 numpy array."""
        img = tfds.core.lazy_imports.PIL_Image.open(data)
        img = np.array(img.getdata()).reshape(img.size).astype(np.float32)
        return img

    def preprocess_image(imgarray): 
        return (imgarray - np.mean(imgarray))/np.std(imgarray)
    image_list = []

    filelist1 = os.listdir(imageband_directory)

    for fpath in filelist1:
        fullpath = imageband_directory+"/"+fpath

        if fullpath.endswith(('B02.tif', 'B03.tif', 'B04.tif')):
            imgarray = _load_tif(imageband_directory+"/"+fpath)
            image_list.append(preprocess_image(imgarray))

        if fullpath.endswith('B05.tif'):
            vegetation_red_edge1 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B06.tif'):
            vegetation_red_edge2 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B07.tif'):
            vegetation_red_edge3 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B11.tif'):
            SWIR_1 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B12.tif'):
            SWIR_2 = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B8A.tif'):
            narrow_nir = _load_tif(imageband_directory+"/"+fpath)

        if fullpath.endswith('B08.tif'):
            nir = _load_tif(imageband_directory+"/"+fpath)

    mean_vegetation = (vegetation_red_edge1 + vegetation_red_edge2 + vegetation_red_edge3)/3

    mean_swir = (SWIR_1 + SWIR_2)/2

    mean_vegetation = np.repeat(np.repeat(mean_vegetation, 2, axis=1), 2, axis=0)
    
    mean_swir = np.repeat(np.repeat(mean_swir,2,axis=1),2,axis=0)

    narrow_nir = np.repeat(np.repeat(narrow_nir,2,axis=1),2,axis=0)

    mean_nir = (narrow_nir + nir)/2

    image_list.append(preprocess_image(mean_vegetation))
    image_list.append(preprocess_image(mean_swir))
    image_list.append(preprocess_image(mean_nir))

    X = np.stack(image_list, axis=2)   # to get (height,width,3)

    # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
    X = np.expand_dims(X, axis=0)

    X = np.array(X, dtype=np.float32)  # Final shape for onnx runtime.
    
    return X

In [7]:
# # Create complete list of file names
# forestfilenames = ["competition_data/trainingdata/forest/" +
#                    x for x in os.listdir("competition_data/trainingdata/forest")]
# nonforestfilenames = ["competition_data/trainingdata/nonforest/" +
#                       x for x in os.listdir("competition_data/trainingdata/nonforest")]
# otherfilenames = ["competition_data/trainingdata/other/" +
#                   x for x in os.listdir("competition_data/trainingdata/other")]

# filenames = forestfilenames+nonforestfilenames+otherfilenames

# # preprocess rbg images into 120,120,3 numpy ndarray
# preprocessed_image_data = []
# for i in filenames:
#     try:
#         preprocessed_image_data.append(preprocessor(i))
#     except:
#         pass

In [8]:
# np.save("6_Channel_PreProcessedImages_March21st",preprocessed_image_data)

In [9]:
preprocessed_image_data = np.load("6_Channel_PreProcessedImages_March21st.npy")

In [10]:
# Set up y data
from itertools import repeat
forest=repeat("forest",5000)
nonforest=repeat("nonforest",5000)
other=repeat("snow_shadow_cloud",5000)
ylist=list(forest)+list(nonforest)+list(other)

In [11]:
# Shuffle X and y data
from sklearn.utils import shuffle
X_train, y_train = shuffle(preprocessed_image_data, ylist, random_state=0)

In [12]:
X_train=np.vstack(X_train) # convert X from list to array

In [13]:
X_train.shape

(15000, 120, 120, 6)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet import ResNet101
import tensorflow as tf
import pandas 


In [20]:
# Run using GPU
with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
    model = Sequential()
    model.add(ResNet101(include_top=False , weights=None ,input_shape=(120,120,6)))
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(2048, activation='tanh')) 
    model.add(BatchNormalization()) # For Next Model 
    model.add(Dropout(0.2))
    
    model.add(Dense(2048, activation='tanh'))
    model.add(BatchNormalization()) # For Next Model 
    model.add(Dropout(0.2))
    
    model.add(Dense(512, activation='tanh')) 
    model.add(Dropout(0.2)) # For Next Model 

    model.add(Dense(256, activation='tanh')) 
    model.add(Dropout(0.2)) # For Next Model 

    
    model.add(Dense(3, activation='softmax')) # Last fully-connected layer of 3 outputs (3 categories).
    model.summary()
    
    
    
    # Compile model...
    model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint("March21_st_2PM_ResNet6_Channel_101",
                                            save_best_only=True)
    
    
    model.fit(  X_train, 
                pandas.get_dummies(y_train),
                validation_split = .3, 
                epochs = 10,
                callbacks=[checkpoint])
    

    model.save("March21_st_2PM_6_ChannelResNet_101.h5")


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 4, 4, 2048)        42667584  
                                                                 
 global_average_pooling2d_4   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_20 (Dense)            (None, 2048)              4196352   
                                                                 
 batch_normalization_8 (Batc  (None, 2048)             8192      
 hNormalization)                                                 
                                                                 
 dropout_16 (Dropout)        (None, 2048)              0         
                                                                 
 dense_21 (Dense)            (None, 2048)             

2022-03-21 19:55:49.454842: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3628800000 exceeds 10% of free system memory.


Epoch 1/10
329/329 [==============================] - ETA: 0s - loss: 1.0592 - accuracy: 0.5049

INFO:tensorflow:Assets written to: March21_st_2PM_ResNet6_Channel_101/assets


INFO:tensorflow:Assets written to: March21_st_2PM_ResNet6_Channel_101/assets


...

INFO:tensorflow:Assets written to: March21_st_2PM_ResNet6_Channel_101/assets


329/329 [==============================] - 143s 436ms/step - loss: 0.5184 - accuracy: 0.7803 - val_loss: 0.5088 - val_accuracy: 0.7960
Epoch 7/10
329/329 [==============================] - 95s 289ms/step - loss: 0.4890 - accuracy: 0.8035 - val_loss: 0.5741 - val_accuracy: 0.7751
Epoch 8/10
329/329 [==============================] - ETA: 0s - loss: 0.4402 - accuracy: 0.8210

INFO:tensorflow:Assets written to: March21_st_2PM_ResNet6_Channel_101/assets


INFO:tensorflow:Assets written to: March21_st_2PM_ResNet6_Channel_101/assets


329/329 [==============================] - 142s 431ms/step - loss: 0.4402 - accuracy: 0.8210 - val_loss: 0.4811 - val_accuracy: 0.8024
Epoch 9/10
329/329 [==============================] - 95s 288ms/step - loss: 0.4273 - accuracy: 0.8266 - val_loss: 0.7465 - val_accuracy: 0.7531
Epoch 10/10
329/329 [==============================] - 95s 289ms/step - loss: 0.4278 - accuracy: 0.8288 - val_loss: 0.5007 - val_accuracy: 0.8156


In [15]:
model = tf.keras.models.load_model('March21_st_2PM_6_ChannelResNet_101.h5')
# model.load_weights("March21_st_2PM_ResNet6_Channel_101")


2022-03-21 20:57:49.660028: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 20:57:49.744340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 20:57:49.744870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 20:57:49.745967: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [16]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("March21_st_2PM_ResNet6_Channel_101", save_best_only=True)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6), metrics=['accuracy'])


model.fit(X_train, pandas.get_dummies(y_train),validation_split = .3, epochs = 1,callbacks=[checkpoint])
# model.save("March21_st_2PM_6_ChannelResNet_101.h5")

2022-03-21 20:57:57.016105: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3628800000 exceeds 10% of free system memory.
2022-03-21 20:58:00.623204: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3628800000 exceeds 10% of free system memory.
2022-03-21 20:58:14.374154: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-03-21 20:58:14.717657: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-03-21 20:58:15.357252: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could

328/329 [============================>.] - ETA: 0s - loss: 0.3026 - accuracy: 0.8812

2022-03-21 20:59:39.714307: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-03-21 20:59:39.721956: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-03-21 20:59:39.734007: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-03-21 20:59:39.737991: W tensorflow/core/common_runtime/bfc_allocato

329/329 [==============================] - ETA: 0s - loss: 0.3031 - accuracy: 0.8810

2022-03-21 20:59:40.035797: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1555200000 exceeds 10% of free system memory.
2022-03-21 20:59:41.582145: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1555200000 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: March21_st_2PM_ResNet6_Channel_101/assets


INFO:tensorflow:Assets written to: March21_st_2PM_ResNet6_Channel_101/assets


329/329 [==============================] - 157s 439ms/step - loss: 0.3031 - accuracy: 0.8810 - val_loss: 0.4187 - val_accuracy: 0.8324


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=9e-8), metrics=['accuracy'])


model.fit(X_train, pandas.get_dummies(y_train), epochs = 5,callbacks=[checkpoint])

In [ ]:
# model.load_weights("March21_st_2PM_ResNet6_Channel_101")

model.save("March21_st_2PM_6_ChannelResNet_101.h5")

In [ ]:
print("Did it run ")

#### Save preprocessor function to local "preprocessor.zip" file

In [ ]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

#### Save model to local ".onnx" file

In [ ]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [ ]:
# import and preprocess X_test images in correct order...
# ...for leaderboard prediction submissions
filenumbers = [str(x) for x in range(1, 5001)]
filenames = ["competition_data/testdata/test/test"+x for x in filenumbers]

# preprocess rbg images into 120,120,3 numpy ndarray
preprocessed_image_data = []
for i in filenames:
    try:
        preprocessed_image_data.append(preprocessor(i))
    except:
        pass

In [ ]:
X_test=np.vstack(preprocessed_image_data) 


In [ ]:
X_test.shape

In [ ]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

# Note -- This is the unique rest api that powers this climate change image classification  Model Plaground
# ... Update the apiurl if submitting to a new competition

apiurl="https://srdmat3yhf.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=apiurl)

In [42]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [44]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [pandas.get_dummies(y_train).columns[i] for i in prediction_column_index]

157/157 [==============================] - 7s 44ms/step


In [46]:
np.save("March21_st_2PM_ResNet6_Channel_Predictions",prediction_labels)

In [48]:
# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

 18% [.........                                             ] 1613824 / 8938820

100% [......................................................] 8938820 / 8938820

/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:287: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  leaderboard = leaderboard.append(metadata, ignore_index=True, sort=False)
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:289: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  leaderboard['username']=leaderboard.pop("username")
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:290: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=

100% [........................................................] 775866 / 775866

/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:287: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  leaderboard = leaderboard.append(metadata, ignore_index=True, sort=False)
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:289: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  leaderboard['username']=leaderboard.pop("username")
/home/ecbm4040/climate_change/lib/python3.8/site-packages/aimodelshare/model.py:290: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=

Insert search tags to help users find your model (optional): ResNet-Architecture_withStandardizedPreprocessing
Provide any useful notes about your model (optional): ResNet-Architecture_withStandardizedPreprocessing , More Dense Models after ResNet Convolution

Your model has been submitted as model version 120

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1535


In [49]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

## 5. Repeat submission process to improve place on leaderboard

*Train and submit your own models using code modeled after what you see above.*

It may also be useful to examine the architeture of models that perform particuarly well/poorly, or to compare models you've created with similar models submitted by others. Use the compare_models function in combination with the leaderboard to learn more about models that been previously submitted and potentially make decisiona about what you should do next.

In [ ]:
# Compare two or more models
data=mycompetition.compare_models([1, 5], verbose=1)
mycompetition.stylize_compare(data)